In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import logging
import ibis as ib
import geopandas as gpd
import nird.road_revised as func
from nird.utils import load_config
import json

ib.options.interactive = True

base_path = Path(load_config()['paths']['soge_clusters'])
warnings.simplefilter('ignore')
tqdm.pandas()

In [10]:
base_path

PosixPath('/soge-home/projects/mistral/DAFNI_NIRD/processed_data')

In [4]:
# disrupted_od = ib.read_parquet(base_path.parent / "results" / "disruption_analysis" / "revision" / "ibis_results" / "disrupted_od_depth15_event5_day0.pq")

In [15]:
def main(
    depth_key,
    event_key,
    number_of_cpu,
):
    # Load network parameters
    with open(base_path / "parameters" / "flow_breakpoint_dict.json", "r") as f:
        flow_breakpoint_dict = json.load(f)
        
    for day in range(0, 1): # 0-12
        logging.info("Loading inputs...")
        road_links = ib.read_parquet(base_path.parent / "results" / "disruption_analysis" / "revision" / "ibis_results" / f"road_links_depth{depth_key}_event{event_key}_day{day}.gpq")
        road_links = road_links.cast({'geometry' : 'GEOMETRY'}).execute().set_geometry('geometry').set_crs('epsg:27700')
        if day == 0: # event_day
            road_links['acc_speed'] = road_links[['acc_speed', 'max_speed']].min(axis=1) 
        disrupted_od = pd.read_parquet(base_path.parent / "results" / "disruption_analysis" / "revision" / "ibis_results" / f"disrupted_od_depth{depth_key}_event{event_key}_day{day}.pq")
        disrupted_od.flow = disrupted_od.flow.fillna(0)
        disrupted_od["Car21"] = (disrupted_od.flow - disrupted_od.min_link_capacity).clip(lower=0)
        disrupted_od.Car21 = disrupted_od.Car21.round(0).astype(int)
        
        total_cost = (
            disrupted_od.Car21 * 
            (
                disrupted_od.operating_cost_per_flow +
                disrupted_od.time_cost_per_flow +
                disrupted_od.toll_cost_per_flow
            )
        ).sum()
        
        logging.info("Creating network...")
        valid_road_links = road_links[(road_links.acc_capacity > 0) & (road_links.acc_speed >0)].reset_index(drop=True)
        network, valid_road_links = func.create_igraph_network(valid_road_links)

        logging.info("Running flow simulation...")
        logging.info(f"The total disrupted flow is: {disrupted_od.Car21.sum()}")
        valid_road_links, isolation, _, (_,_,_,total_cost2) = func.network_flow_model(
            valid_road_links,
            network,
            disrupted_od[["origin_node", "destination_node", "Car21"]],
            flow_breakpoint_dict,
            num_of_cpu = number_of_cpu,
        )
        logging.info(f"The total pre-event cost: £ million {total_cost/1e6}")
        logging.info(f"The total after-event cost: £ million {total_cost2/1e6}")
        rerouting_cost = total_cost2 - total_cost
        logging.info(f"The rerouting cost for depth{depth_key}-event{event_key}-scenario{day}: £{rerouting_cost}")
        logging.info("Saving results to disk...")
        out_path = (
            base_path.parent
            / "results"
            / "rerouting_analysis"
            / "revision"
            / str(depth_key)  # e.g.,30
            / str(event_key)  # e.g.,17
        )
        out_path.mkdir(parents=True, exist_ok=True)
        # isolations
        isolation_df = pd.DataFrame(isolation, columns = ["origin_node", "destination_node", "Car21"])
        isolation_df.to_csv(out_path / f"trip_isolations_{day}.csv", index=False)
        # edge flows
        road_links = road_links.set_index("e_id")
        road_links.update(valid_road_links.set_index("e_id")["acc_flow"])
        road_links.reset_index(drop = True, inplace = True)
        road_links["change_flow"] = road_links.acc_flow - road_links.current_flow
        road_links.to_parquet(out_path / f"edge_flows_{day}.gpq", engine="pyarrow")

        del valid_road_links
        del road_links
        del disrupted_od
    
    # rerouting costs over all scenarios (days)
    cDict[day] = rerouting_cost
    cost_df = pd.DataFrame.from_dict(cDict, orient="index", columns=["rerouting_cost"]).reset_index()
    cost_df.rename(columns = {"index": "scenario"}, inplace =True)
    cost_df.to_csv(out_path / "rerouting_cost.csv", index=False)

if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s %(process)d %(filename)s %(message)s", level=logging.INFO
    )
    main(depth_key =15, event_key=1, number_of_cpu=20)

2025-09-18 11:10:11,097 1951255 2178983545.py Loading inputs...
2025-09-18 11:10:23,588 1951255 2178983545.py Creating network...
2025-09-18 11:11:46,734 1951255 2178983545.py Running flow simulation...
2025-09-18 11:11:46,737 1951255 2178983545.py The total disrupted flow is: 380367
2025-09-18 11:11:46,753 1951255 road_revised.py The initial supply is 380367
2025-09-18 11:11:46,949 1951255 road_revised.py The initial number of edges in the network: 501468
2025-09-18 11:11:46,981 1951255 road_revised.py The initial number of origins: 52215
2025-09-18 11:11:47,008 1951255 road_revised.py The initial number of destinations: 34122
2025-09-18 11:11:47,010 1951255 road_revised.py No.1 iteration starts:
2025-09-18 11:11:47,625 1951255 road_revised.py Initial isolated flows: 13117
Creating argument list: 100%|██████████| 51274/51274 [06:38<00:00, 128.74it/s]
2025-09-18 11:18:34,967 1951255 road_revised.py Completed 0 of 51274, 0.00%
2025-09-18 11:25:16,544 1951255 road_revised.py Completed 10

No low saturation edges found.


2025-09-18 11:49:16,707 1951255 road_revised.py Updating edge speeds: 
100%|██████████| 501468/501468 [00:06<00:00, 79526.44it/s]
2025-09-18 11:49:23,454 1951255 road_revised.py The total remain flow (after adjustment) is: 88.0.
2025-09-18 11:49:23,471 1951255 road_revised.py Stop: 99.97370328128011% of flows (exc. isolations) have been allocated and there is no edge overflow!
2025-09-18 11:49:23,652 1951255 road_revised.py The flow simulation is completed!
2025-09-18 11:49:23,654 1951255 road_revised.py total travel cost is (£): 678144797.0722806
2025-09-18 11:49:23,655 1951255 road_revised.py total time-equiv cost is (£): 671456218.8363543
2025-09-18 11:49:23,655 1951255 road_revised.py total operating cost is (£): 6220222.835926468
2025-09-18 11:49:23,656 1951255 road_revised.py total toll cost is (£): 468355.4
2025-09-18 11:49:26,213 1951255 2178983545.py The total pre-event cost: £ million 6.625471997316818
2025-09-18 11:49:26,220 1951255 2178983545.py The total after-event cost: 

NameError: name 'cDict' is not defined